In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
df_train = pd.read_csv('../input/train.csv')

In [ ]:
df_train.head()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline 

In [ ]:
df_train['target'].value_counts().plot(kind="bar", figsize = (20,8))
plt.xlabel('CLASSES')
plt.ylabel('COUNTS')
plt.title("DIFFERENT CLASS TYPE OF TARGET VARIABLE")

In [ ]:
df_train['Length'] = df_train['ciphertext'].str.len()

In [ ]:
df_train.head(n=20)

In [ ]:
df_train.describe()

In [ ]:
df_train['difficulty'].value_counts().plot(kind="bar", figsize = (20,8))
plt.xlabel('CLASSES')
plt.ylabel('COUNTS')
plt.title("DIFFERENT CLASS TYPE OF TARGET VARIABLE")

In [ ]:
print(df_train['difficulty'].value_counts())

In [ ]:
train_diff1 = df_train[df_train['difficulty'] == 1]
train_diff2 = df_train[df_train['difficulty'] == 2]
train_diff3 = df_train[df_train['difficulty'] == 3]
train_diff4 = df_train[df_train['difficulty'] == 4]

train_diff1 = train_diff1.drop(['Id','difficulty'], axis = 1)
train_diff2 = train_diff2.drop(['Id','difficulty'], axis = 1)
train_diff3 = train_diff3.drop(['Id','difficulty'], axis = 1)
train_diff4 = train_diff4.drop(['Id','difficulty'], axis = 1)

train_diff1['ciphertext'] = train_diff1['ciphertext'].apply(lambda x: x.replace('1', ' '))
train_diff2['ciphertext'] = train_diff2['ciphertext'].apply(lambda x: x.replace('8', ' '))
train_diff3['ciphertext'] = train_diff3['ciphertext'].apply(lambda x: x.replace('8', ' '))
train_diff4['ciphertext'] = train_diff4['ciphertext'].apply(lambda x: x.replace('8', ' '))

print(len(train_diff1), len(train_diff2), len(train_diff3), len(train_diff4))

In [ ]:
train_diff = pd.concat([train_diff1, train_diff2, train_diff3, train_diff4])

In [ ]:
from keras.utils import np_utils
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, LSTM, Conv1D, MaxPooling1D, Dropout, Activation, CuDNNLSTM
from keras.layers.embeddings import Embedding
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [ ]:
max_len_text = 300
embedding_dims = 300
vocabulary_size = 20000

# tokenizer = Tokenizer(filters=None, char_level=True, split=' ')
tokenizer = Tokenizer(char_level=True)
tokenizer.fit_on_texts(train_diff['ciphertext'])

In [ ]:
sequences = tokenizer.texts_to_sequences(train_diff['ciphertext'])
data = pad_sequences(sequences, maxlen=max_len_text)

In [ ]:
dummy_y = np_utils.to_categorical(train_diff['target'])

In [ ]:
num_units = 256

model_lstm = Sequential()
model_lstm.add(Embedding(vocabulary_size, embedding_dims, input_length=max_len_text))
model_lstm.add(CuDNNLSTM(num_units))
model_lstm.add(Dense(dummy_y.shape[1], activation='softmax'))
model_lstm.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
batch_size = 32
# num_docs = 20000
num_docs = data.shape[0]

model_lstm.fit(data[:num_docs], dummy_y[:num_docs], validation_split=0.15, batch_size=batch_size, epochs=5)

In [ ]:
df_test = pd.read_csv('../input/test.csv')

In [ ]:
print(df_test['difficulty'].value_counts())

In [ ]:
test_diff1 = df_test[df_test['difficulty'] == 1]
test_diff2 = df_test[df_test['difficulty'] == 2]
test_diff3 = df_test[df_test['difficulty'] == 3]
test_diff4 = df_test[df_test['difficulty'] == 4]

test_diff1 = test_diff1.drop(['Id','difficulty'], axis = 1)
test_diff2 = test_diff2.drop(['Id','difficulty'], axis = 1)
test_diff3 = test_diff3.drop(['Id','difficulty'], axis = 1)
test_diff4 = test_diff4.drop(['Id','difficulty'], axis = 1)

test_diff1['ciphertext'] = test_diff1['ciphertext'].apply(lambda x: x.replace('1', ' '))
test_diff2['ciphertext'] = test_diff2['ciphertext'].apply(lambda x: x.replace('8', ' '))
test_diff3['ciphertext'] = test_diff3['ciphertext'].apply(lambda x: x.replace('8', ' '))
test_diff4['ciphertext'] = test_diff4['ciphertext'].apply(lambda x: x.replace('8', ' '))

print(len(test_diff1), len(test_diff2), len(test_diff3), len(test_diff4))

In [ ]:
test_diff = pd.concat([test_diff1, test_diff2, test_diff3, test_diff4])

In [ ]:
test_sequences = tokenizer.texts_to_sequences(test_diff['ciphertext'])
test_data = pad_sequences(test_sequences, maxlen=max_len_text)

In [ ]:
test_predictions = model_lstm.predict(test_data)

In [ ]:
print(test_data.shape, test_predictions.shape)

In [ ]:
test_ID = df_test['Id']

In [ ]:
final_predictions = np.argmax(test_predictions, axis=1)

In [ ]:
sub = pd.DataFrame()
sub['Id'] = test_ID
sub['Predicted'] = final_predictions
sub.to_csv('submission.csv',index=False)

In [ ]:
os.listdir('.')

In [ ]:
df = pd.read_csv('submission.csv')

In [ ]:
df.head(n=10)